In [16]:
import os
import re
import string
import random
import pyodbc
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from datetime import datetime, timedelta
from vn_fullname_generator import generator
from concurrent.futures import ThreadPoolExecutor, as_completed

## I. Transform

#### Các cài đặt ban đầu

In [17]:
RANDOM_SEED = 42
NUM_ROWS = 10**6
NUM_WORKERS = 16

In [18]:
load_dotenv()

True

Kết nối với SQL Server

In [19]:
def get_db_connection(DB_NAME:str):
    conn = pyodbc.connect(
        f"DRIVER={os.getenv('DB_DRIVER')};"
        f"SERVER={os.getenv('DB_SERVER')};"
        f"DATABASE={DB_NAME};"
        f"UID={os.getenv('DB_UID')};"
        f"PWD={os.getenv('DB_PWD')};"
        f"TrustServerCertificate={os.getenv('TRUST_SERVER_CERTIFICATE')};"
    )
    
    return conn

In [20]:
file_name = {
    'role': 'role.csv',
    'permission': 'permission.csv',
    'role_permission': 'role_permission.csv',
}

dataframes = {
    name: pd.read_csv(f'./data/{filename}', encoding='utf-8')
    for name, filename in file_name.items()
}

In [21]:
role_df = dataframes['role']
permission_df = dataframes['permission']
role_permission_df = dataframes['role_permission']

### 1. Tạo dataframe `account` (id, username, password, full_name, email, status, created_at, updated_at)

Chuẩn hóa câu tiếng Việt về dạng tiếng Anh

In [22]:
def normalize_vietnamese_string(s: str):
    vietnamese_chars = {
        'à': 'a', 'á': 'a', 'ả': 'a', 'ã': 'a', 'ạ': 'a',
        'ă': 'a', 'ằ': 'a', 'ắ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ặ': 'a',
        'â': 'a', 'ầ': 'a', 'ấ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ậ': 'a',
        'đ': 'd',
        'è': 'e', 'é': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ẹ': 'e',
        'ê': 'e', 'ề': 'e', 'ế': 'e', 'ể': 'e', 'ễ': 'e', 'ệ': 'e',
        'ì': 'i', 'í': 'i', 'ỉ': 'i', 'ĩ': 'i', 'ị': 'i',
        'ò': 'o', 'ó': 'o', 'ỏ': 'o', 'õ': 'o', 'ọ': 'o',
        'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ổ': 'o', 'ỗ': 'o', 'ộ': 'o',
        'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ở': 'o', 'ỡ': 'o', 'ợ': 'o',
        'ù': 'u', 'ú': 'u', 'ủ': 'u', 'ũ': 'u', 'ụ': 'u',
        'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ử': 'u', 'ữ': 'u', 'ự': 'u',
        'ỳ': 'y', 'ý': 'y', 'ỷ': 'y', 'ỹ': 'y', 'ỵ': 'y'
    }

    s = s.lower()

    for vn_char, latin_char in vietnamese_chars.items():
        s = s.replace(vn_char, latin_char)
    
    s = re.sub(r'[^a-z ]','', s)

    return s

Tạo email ngẫu nhiên

In [23]:
def generate_email(name: str ):

    normalized = normalize_vietnamese_string(name)
    words = normalized.split()

    email_format = random.choice([
        f'{words[-1]}.{words[0]}',
        f'{words[0]}.{words[-1]}',
        f'{words[0]}{words[-1]}',
        f'{words[-1]}{words[0]}',
        f'{words[0][0]}{words[-1]}',
        f'{words[-1]}{words[0][0]}',
        f'{words[0]}{random.randint(1, 999)}',
        f'{words[-1]}{random.randint(1, 999)}',
        f'{words[0]}_{random.randint(1, 999)}',
        f'{words[-1]}_{random.randint(1, 999)}',
    ])

    if random.random() < 0.3:
        email_format += str(random.randint(1,999))
    
    domain = random.choice(['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'icloud.com', 'gdscptit.dev', 'ptit.edu.vn', 'stu.ptit.edu.vn'])

    return f'{email_format}@{domain}'

Tạo số điện thoại ngẫu nhiên

In [24]:
def generate_phone_number():    
    prefixes = ['032', '033', '034', '035', '036', '037', '038', '039', '096', '097', '098', '086',
                '088', '091', '094', '081', '082', '083', '084', '085',
                '070', '079', '077', '076', '078', '090', '092', '089'
                ]
    
    prefix = random.choice(prefixes)

    remaining = ''.join(str(random.randint(0,9)) for _ in range(7))

    return prefix + remaining

Tạo xâu ngẫu nhiên (phục vụ tạo `username` và `password` sau này)

In [25]:
def generate_random_string(length=8, use_special=False):
    chars = string.ascii_letters + string.digits
    if use_special:
        safe_punctiations = '!@#$%^^&*_-'
        chars += safe_punctiations
    return ''.join(random.choices(chars, k=length))

Tạo ngày ngẫu nhiên

In [26]:
def generate_random_date(start, end):
    
    start_date = datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.strptime(end, "%Y-%m-%d")

    random_days = random.randint(0, (end_date - start_date).days)

    random_date = start_date + timedelta(days=random_days)

    return random_date.strftime("%Y-%m-%d")

Tạo `status` tài khoản ngẫu nhiên với 3 trạng thái `active`, `inactive`, `banned` (trong đó xác suất sinh ra `banned` thấp hơn 2 trạng thái còn lại)

In [27]:
def generate_random_user_status():
    status = ['active', 'inactive', 'banned']
    probability = [0.65, 0.25, 0.1]
    return random.choices(status, weights=probability, k=1)[0]

Tạo địa chỉ ngẫu nhiên (Sử dụng database lưu trữ các khu vực hành chính của Việt Nam)

In [28]:
def fetch_addresses():
    address_db = os.getenv('DB_ADDRESS')
    conn = get_db_connection(address_db)
    df = pd.read_sql("SELECT * FROM address", conn)
    conn.close()
    return df

def generate_random_address(address_df):
    row = address_df.sample(n=1).iloc[0]
    ward = row['wards']
    district = row['districts']
    province = row['provinces']

    return f"{ward}, {district}, {province}"

Tạo tài khoản ngẫu nhiên

In [29]:
email_used = set()


def generate_account(id):
    try:

        username = generate_random_string(length=random.randint(5, 10))
        password = generate_random_string(length=random.randint(8, 12), use_special=True)

        gender = random.randint(0, 1)
        name = generator.generate(gender)
        
        while True:
            email = generate_email(name)
            if email not in email_used:
                email_used.add(email)
                break

        status = generate_random_user_status()

        create_at = generate_random_date("2023-01-01", datetime.now().strftime("%Y-%m-%d"))
        update_at = generate_random_date(create_at, datetime.now().strftime("%Y-%m-%d"))


        return {
            'id': id,
            'username': username,
            'password': password,
            'name': name,
            'email': email,
            'status': status,
            'created_at': create_at,
            'updated_at': update_at
        }
    
    except Exception as e:
        print('Error generating user: ', e)
        return None


Sử dụng ThreadPoolExecutor để tạo nhiều luồng sinh tài khoản, tăng tốc độ xử lý khi phải sinh dữ liệu với số lượng lớn

In [30]:
def generate_accounts(n, max_workers=16):
    data = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(generate_account, i) for i in range(n)]
        for f in tqdm (as_completed(futures), total=n, desc="Generating account rows", unit="record", colour="green"):
            result = f.result()
            if result:
                data.append(result)
    
    return pd.DataFrame(data)

In [31]:
random.seed(RANDOM_SEED)

print('Create accounts...')
print('----------------------------------')
account_df = generate_accounts(n=NUM_ROWS, max_workers=NUM_WORKERS)
print('Create accounts done!')
print('----------------------------------')


Create accounts...
----------------------------------


Generating account rows: 100%|██████████| 1000000/1000000 [03:48<00:00, 4371.86record/s]


Create accounts done!
----------------------------------


In [32]:
account_df.sort_values(by='id', inplace=True)
account_df.reset_index(drop=True, inplace=True)

account_df.head()

,id,username,password,name,email,status,created_at,updated_at
0,0,gTpigTHKbf,KboVNqR7a,Ngô Nhã Lý,lyn390@yahoo.com,active,2023-12-19,2024-10-23
1,1,YTH8xIZ,#AofW4_!@BH8,Đỗ Nhật Lệ,do.le@outlook.com,active,2023-09-08,2024-02-22
2,2,x9NIQ0Wo,6De%P0pK,Hoàng Lâm Trường,hoang_763599@ptit.edu.vn,inactive,2024-02-13,2024-08-16
3,3,9iEU1j,5EeB-M*!a,Trương Minh Nhi,tnhi@stu.ptit.edu.vn,active,2025-01-09,2025-05-01
4,4,5UQSyP,FLiqyQqqfU,Lê Bạch Mai,maile498@hotmail.com,active,2025-01-10,2025-05-02


### 2.Tạo dataframe `customer` và `manager`

Phân bổ tài khoản cho `customer` và `manager` (200 tài khoản cho manager và phần còn lại cho customer)

In [33]:
eligible_accounts = account_df[account_df['status'] != 'banned']

manager_ids = eligible_accounts.sample(n=200, random_state=RANDOM_SEED).index

customer_ids = sorted(account_df.drop(manager_ids).index.to_list())

manager_ids = sorted(manager_ids.to_list())

Tạo dataframe `customer` (id, account_id, phone_number, address)

In [34]:
def generate_customer(id, account_id, address_df):
    
    phone = generate_phone_number()
    address = generate_random_address(address_df)

    return {
        'id': id,
        'account_id': account_id,
        'phone_number': phone,
        'address': address
    }

Tạo dataframe `manager` (id, role_id, account_id)

Tạo role_id ngẫu nhiên cho toàn bộ nhân viên

In [35]:
random.seed(RANDOM_SEED)

role_ids = random.choices(role_df['id'].tolist(), k=len(manager_ids))

In [36]:
def generate_manager(id, account_id, role_id):
    
    return {
        'id': id,
        'account_id': account_id,
        'role_id': role_id,
    }

Dùng ThreadPoolExecutor tạo customers và managers

In [37]:
def generate_customers(address_df, max_workers=16):
    customers = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(generate_customer, i, account_id, address_df) for i, account_id in enumerate(customer_ids)]
        
        for f in tqdm (as_completed(futures), total=len(customer_ids), desc="Generating customer rows", unit="record", colour="green"):
            result = f.result()
            if result:
                customers.append(result)
        
    return pd.DataFrame(customers)

def generate_managers(max_workers=16):
    managers = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(generate_manager, i, account_id, role_id) for i, (account_id, role_id) in enumerate(zip(manager_ids, role_ids))]
        
        for f in tqdm (as_completed(futures), total=len(manager_ids), desc="Generating manager rows", unit="record", colour="green"):
            result = f.result()
            if result:
                managers.append(result)
        
    return pd.DataFrame(managers)

Load dataset các khu vực hành chính

In [38]:
print('Fetching address data...')
print('----------------------------------')
address_df = fetch_addresses()
print('Fetching address data done!')

Fetching address data...
----------------------------------
Fetching address data done!


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3604\3675934813.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM address", conn)


In [39]:
address_df.head()

,id,wards,districts,provinces
0,0,Phường Quang Trung,Thành phố Hà Giang,Tỉnh Hà Giang
1,1,Phường Trần Phú,Thành phố Hà Giang,Tỉnh Hà Giang
2,2,Phường Ngọc Hà,Thành phố Hà Giang,Tỉnh Hà Giang
3,3,Phường Nguyễn Trãi,Thành phố Hà Giang,Tỉnh Hà Giang
4,4,Phường Minh Khai,Thành phố Hà Giang,Tỉnh Hà Giang


Tiến hành tạo dataframe `customer` và `manager`

In [40]:
print('Generate customer data...')
print('----------------------------------')
customer_df = generate_customers(address_df, max_workers=NUM_WORKERS)
print('Generate customer data done!')
print('----------------------------------')

print('Generate manager data...')
print('----------------------------------')
manager_df = generate_managers(max_workers=NUM_WORKERS)
print('Generate manager data done!')
print('----------------------------------')

Generate customer data...
----------------------------------


Generating customer rows: 100%|██████████| 999800/999800 [11:05<00:00, 1501.37record/s]


Generate customer data done!
----------------------------------
Generate manager data...
----------------------------------


Generating manager rows: 100%|██████████| 200/200 [00:00<00:00, 93155.00record/s]

Generate manager data done!
----------------------------------


In [41]:
customer_df.sort_values(by='id', inplace=True)
customer_df.reset_index(drop=True, inplace=True)

manager_df.sort_values(by='id', inplace=True)
manager_df.reset_index(drop=True, inplace=True)

In [42]:
customer_df.head()

,id,account_id,phone_number,address
0,0,0,0343267736,"Phường Phong Hải, Thị xã Quảng Yên, Tỉnh Quảng..."
1,1,1,0332606474,"Xã Thạnh Bắc, Huyện Tân Biên, Tỉnh Tây Ninh"
2,2,2,0918723430,"Xã Thanh Xuân, Huyện Thanh Hà, Tỉnh Hải Dương"
3,3,3,0848050097,"Xã Tân Cương, Thành phố Thái Nguyên, Tỉnh Thái..."
4,4,4,0828208121,"Xã Tân Nghĩa, Huyện Cao Lãnh, Tỉnh Đồng Tháp"


In [43]:
manager_df.head()

,id,account_id,role_id
0,0,5015,2
1,1,11459,1
2,2,15265,1
3,3,18994,1
4,4,21352,3


## II. Load

Kết nối với Database

In [44]:
DB_NAME = os.getenv('DB_NAME')
conn = get_db_connection(DB_NAME)
cursor = conn.cursor()

Loại bỏ các bảng nếu tồn tại trong database

In [45]:
cursor.execute("""
    IF OBJECT_ID('customer', 'U') IS NOT NULL DROP TABLE customer;
               
    IF OBJECT_ID('manager', 'U') IS NOT NULL DROP TABLE manager               

    IF OBJECT_ID('account', 'U') IS NOT NULL DROP TABLE account;
""")

In [46]:
cursor.execute("""
    IF OBJECT_ID('role_permission', 'U') IS NOT NULL DROP TABLE role_permission;
    
    IF OBJECT_ID('permission', 'U') IS NOT NULL DROP TABLE permission;
    
    IF OBJECT_ID('role', 'U') IS NOT NULL DROP TABLE [role];
""")

### 1. Lưu dataframe `role`, `permission`, `role_permission` vào SQL Server

Tạo bảng `role`, `permission`, `role_permission`

In [47]:
cursor.execute("""
    CREATE TABLE [role] (
        id INT PRIMARY KEY IDENTITY(1,1),
        name NVARCHAR(255) NOT NULL,
        status NVARCHAR(50) DEFAULT 'Active'
    )
""")

cursor.execute("""
    CREATE TABLE permission (
        id INT PRIMARY KEY IDENTITY(1,1),
        name NVARCHAR(255) NOT NULL,
    )
""")

cursor.execute("""
    CREATE TABLE role_permission (
        role_id INT,
        permission_id INT,
        CONSTRAINT pk_role_permission PRIMARY KEY (role_id, permission_id),
        CONSTRAINT fk_role_permission_role FOREIGN KEY (role_id) REFERENCES [role](id),
        CONSTRAINT fk_role_permission_permission FOREIGN KEY (permission_id) REFERENCES permission(id)
    );
""")

Tạo tuples từ dataframe

In [48]:
role_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in role_df.itertuples(index=False, name=None)
]

permission_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in permission_df.itertuples(index=False, name=None)
]

role_permission_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in role_permission_df.itertuples(index=False, name=None)
]

Chèn dữ liệu vào bảng `role`

In [49]:
cursor.execute("SET IDENTITY_INSERT [role] ON;")
cursor.executemany("INSERT INTO [role] (id, name, status) VALUES (?, ?, ?)", role_tuples)
cursor.execute("SET IDENTITY_INSERT [role] OFF;")

Chèn dữ liệu vào bảng `permission`

In [50]:
cursor.execute("SET IDENTITY_INSERT [permission] ON;")
cursor.executemany("INSERT INTO permission (id, name) VALUES (?, ?)", permission_tuples)
cursor.execute("SET IDENTITY_INSERT [permission] OFF;")

Chèn dữ liệu vào bảng `role_permission`

In [51]:
cursor.executemany("INSERT INTO role_permission (role_id, permission_id) VALUES (?, ?)", role_permission_tuples)

### 2. Lưu dataframe `customer`, `manager`, `account` vào SQL Server

Tạo bảng `customer`, `manager`, `account`

In [52]:
cursor.execute("""
    CREATE TABLE account (
        id INT PRIMARY KEY IDENTITY(1,1),
        username NVARCHAR(255) NOT NULL,
        password NVARCHAR(255) NOT NULL,
        full_name NVARCHAR(255) NOT NULL,
        email NVARCHAR(255) UNIQUE NOT NULL,
        status NVARCHAR(50) DEFAULT 'Active',
        created_at DATETIME DEFAULT GETDATE(),
        updated_at DATETIME
    )
""")

cursor.execute("""
    CREATE TABLE customer (
        id INT PRIMARY KEY IDENTITY(1,1),
        account_id INT NOT NULL,
        phone_number NVARCHAR(15),
        address NVARCHAR(500),
        CONSTRAINT fk_customer_account FOREIGN KEY (account_id) REFERENCES account(id)
    )
""")

cursor.execute("""
    CREATE TABLE manager (
        id INT PRIMARY KEY IDENTITY(1,1),
        account_id INT NOT NULL,
        role_id INT NOT NULL,
        CONSTRAINT fk_manager_role FOREIGN KEY (role_id) REFERENCES role(id),
        CONSTRAINT fk_manager_account FOREIGN KEY (account_id) REFERENCES account(id)
    )
""")

Tạo tuples từ dataframe

In [53]:
account_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(account_df.itertuples(index=False, name=None), desc="Creating account tuples", total=account_df.shape[0], unit="row", colour="green")
]

customer_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(customer_df.itertuples(index=False, name=None), desc="Creating customer tuples", total=customer_df.shape[0], unit="row", colour="green")
]

manager_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(manager_df.itertuples(index=False, name=None), desc="Creating manager tuples", total=manager_df.shape[0], unit="row", colour="green")
]

Creating manager tuples: 100%|██████████| 200/200 [00:00<00:00, 155488.56row/s]


Chèn dữ liệu vào bảng `account`

In [54]:
cursor.execute("SET IDENTITY_INSERT account ON")
cursor.fast_executemany = True
cursor.executemany(
    "INSERT INTO account (id, username, password, full_name, email, status, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
    account_tuples
)
cursor.execute("SET IDENTITY_INSERT account OFF")

Chèn dữ liệu vào bảng `customer`

In [55]:
cursor.execute("SET IDENTITY_INSERT customer ON")
cursor.executemany(
    "INSERT INTO customer (id, account_id, phone_number, address) VALUES (?, ?, ?, ?)",
    customer_tuples
)
cursor.execute("SET IDENTITY_INSERT customer OFF")

Chèn dữ liệu vào bảng `manager`

In [56]:
cursor.execute("SET IDENTITY_INSERT manager ON")
cursor.executemany(
    "INSERT INTO manager (id, account_id, role_id) VALUES (?, ?, ?)",
    manager_tuples
)
cursor.execute("SET IDENTITY_INSERT manager OFF")

### 3. Đóng kết nối

In [57]:
conn.commit()
conn.close()